In [1]:
import sys
import subprocess

def install_package(package):
    if isinstance(package, str):
        package = package.split(" ")
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + package)

In [2]:
install_package("trulens trulens-providers-openai trulens-apps-llamaindex llama_index openai")

In [3]:
import json
import pandas as pd

with open("../dataset.json") as f:
    data = json.load(f)

# questions = data['question']
# ground_truths = data['ground_truths']
# answers = data['answer']
# contexts = data['contexts']

dataset = {
    "query": data['question'],
    "response": data['answer'],
    "contexts": data['contexts'],
}
df = pd.DataFrame(dataset)
df.head()

,query,response,contexts
0,What months have the highest and lowest sea-ic...,"According to the documents, March is usually t...",[September is usually the month with the lowes...
1,What are the benefits of using the WAVEFLOW mo...,The WAVEFLOW model is designed to introduce ph...,"[Specifically, the project introduces recent i..."
2,How does the new method handle vertical observ...,The new method handles vertical observations o...,[As one would expect the uncertainty is more s...
3,What are the effects of extreme water levels o...,The effects of extreme water levels on coastal...,[Skip to main content\n#EUSpace\n Media\n Even...
4,What kind of data do Green Ocean models rely on?,Green Ocean models rely on observational data ...,[Skip to main content\n Search only in\n Page\...


In [4]:
from trulens.apps.virtual import VirtualApp

virtual_app = VirtualApp()

In [9]:
import json

TOGETHERAI_API_KEY = ""

def load_env():
    with open('../settings.json') as f:
        settings = json.load(f)
    globals().update(settings)

load_env()
TOGETHERAI_API_KEY = llm_chat_key
assert TOGETHERAI_API_KEY

In [10]:
# from llama_index.llms.openai import OpenAI
# from llama_index.llms.together import TogetherLLM

# llm_model_params = {
#     "api_key": TOGETHER_API_KEY,
#     "base_url": "https://api.together.xyz/v1",
#     "model": "Qwen/Qwen2.5-7B-Instruct-Turbo"
# }

# provider = TogetherLLM(**llm_model_params)
from trulens.providers.openai import OpenAI
llm_model_params = {
    "api_key": TOGETHERAI_API_KEY,
    "base_url": "https://api.together.xyz/v1",
    "model": "Qwen/Qwen2.5-7B-Instruct-Turbo"
}
provider = OpenAI(model_engine=llm_model_params.pop('model'), **llm_model_params)

In [11]:
from trulens.core import Feedback

context = VirtualApp.select_context()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retriever.get_context.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.retriever.get_context.rets[:].collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [12]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession()
run_dashboard(session)

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.100.189:54595 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [13]:
from trulens.apps.virtual import TruVirtual

virtual_recorder = TruVirtual(
    app_name="RAG",
    app_version="simple-v2",
    app=virtual_app,
    feedbacks=[f_context_relevance, f_groundedness, f_qa_relevance],
)

In [14]:
virtual_records = virtual_recorder.add_dataframe(df)

Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.
Could not find an instance of DummyEndpo